# Comparación de grupos
"*La mitad de mi gasto en publicidad no sirve para nada. El problema es que no sé qué mitad" atribuida a John Wanamaker(1838-1922)*

## Pruebas A/B
En el área de negocios, especialmente en innovación digital, es común realizar pruebas A/B para evaluar las respuestas de los usuarios a diferentes alternativas en el diseño de un sitio o una interfaz.  
Entre sus características básicas está que los usuarios son asignados al azar a un grupo de control (A) o a un grupo experimental (B). A cada grupo se les muestra una alternativa diferente y se recopilan diversas métricas. Finalmente, analizando los resultados, se concluye qué alternativa tuvo mejor desempeño.  
En la siguiente liga se muestran varios ejemplos de aplicaciones prácticas:
https://goodui.org/leaks/. Este tipo de experimentos ejemplifica la aplicación de técnicas que veremos en esta sesión relacionadas con la comparación de dos o más grupos

El archivo "cookie_cats.csv" contiene información de 90,189 jugadores. Cuando un jugador instala el juego empieza en el nivel 30, sin embargo, en la empresa que diseñó el juego han propuesto que el jugador empiece en el nivel 40. Para evaluar ambas opciones se diseñó una prueba A/B en la cual los jugadores fueron asignados aleatoriamente para comenzar en el nivel 30 (control) o en el nivel 40 (tratamiento).

<div style="text-align: center;">
<img src="../images/cookie_cats.jpg" alt="Cookie cats" width="300" height="200">
</div>

Las variables en el dataset son:

- userid: identificador del jugador.
- version: gate30, empieza en el nivel 30; gate40: empieza en el nivel 40.
- sum_gamerounds: número de rondas jugadas durante los primeros 14 días.
- retention_1: el jugador jugó un día después de instalar el juego.
- retention_7_ el jugador jugó después de 7 días de haber instalado el juego.

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('https://github.com/adan-rs/AnalisisDatos/raw/main/datasets/cookie_cats.csv')
df.info()

In [ ]:
df.describe().T

In [ ]:
# Eliminación de datos atípicos
from sklearn.ensemble import IsolationForest

def remove_outliers_iso_forest(df, columns, contamination=0.05, random_state=42):
    """
    Elimina valores atípicos utilizando el algoritmo Isolation Forest.
    Retorna DataFrame sin valores atípicos.
    """
    # Initialize and fit Isolation Forest model
    iso_forest = IsolationForest(contamination=contamination, random_state=random_state)
    iso_forest.fit(df[columns])
    
    # Predict labels: 1 (normal) or -1 (outlier)
    labels = iso_forest.predict(df[columns])
    
    # Calculate and display number of excluded outliers
    df_clean = df[labels == 1]
    excluded_values = len(df) - len(df_clean)
    print(f"\nCantidad de valores atípicos excluidos: {excluded_values}")
    return df_clean

In [ ]:
df = remove_outliers_iso_forest(df, columns=['sum_gamerounds'])
df.info()

## Prueba z para dos proporciones
Realizaremos una prueba z para comparar dos proporciones. Las pruebas de hipótesis pueden ser bilaterales (de dos colas) o unilaterales (de una cola). Utilizaremos una prueba bilateral, por lo tanto, las hipótesis se plantean como:
- H<sub>0</sub>: p=p<sub>0</sub> (Hipótesis nula)
- H<sub>1</sub>: p ≠ p<sub>0</sub> (Hipótesis alternativa)

Para calcular las proporciones conviene obtener una tabla con el conteo de cada caso. Realicemos una tabla de contigencia o tabla cruzada. 

In [ ]:
from statsmodels.stats.proportion import proportions_ztest

def compare_proportions(data, group_column, category_column,
                       success_category=None, alpha = 0.05):
    """
    Realiza e interpreta una prueba z bilateral para comparar proporciones entre grupos.
    Args:
        data: DataFrame con los datos
        group_column: Nombre de la columna que identifica los grupos
        category_column: Nombre de la columna con las categorías a comparar
        success_category: Categoría a considerar como "éxito" (si es None, 
                         usa la primera categoría encontrada)
        alpha: Nivel de significancia (predeterminado: 0.05)    
    Returns:
        p-valor
    """
    # Identificar categorías
    categorias = data[category_column].unique()  # Obtener valores únicos
    if len(categorias) != 2:                     # Sólo se comparan dos grupos
        raise ValueError(f"La columna {category_column} debe tener exactamente dos categorías")
    
    # Determinar categoría de éxito
    if success_category is None:
        success_category = categorias[0]
    elif success_category not in categorias:
        raise ValueError(f"La categoría '{success_category}' no existe en los datos")
    
    # Convertir a binario (1 para success_category, 0 para la otra)
    data_binaria = data.copy()
    data_binaria[category_column] = (data[category_column] == success_category).astype(int)
    
    # Calcular estadísticos
    conteos = data_binaria.groupby(group_column)[category_column].agg(['sum', 'count'])
    proporciones = (conteos['sum'] / conteos['count']).round(4)
    
    # Realizar prueba z
    stat, p_value = proportions_ztest(conteos['sum'], conteos['count'])
    
    # Preparar interpretación
    es_significativo = p_value < alpha
    interpretacion = (f"Estadístico z: {stat:.4f}\n"
                      f"Valor p: {p_value:.4f}\n\n"
                      f"Proporciones de '{success_category}' por grupo:\n")
    for grupo in proporciones.index:
        n = conteos.loc[grupo, 'count']
        prop = proporciones[grupo]
        interpretacion += f"{grupo}: {prop:.1%} (n={n})\n"    
    interpretacion += f"\nConclusión: {'Se rechaza' if es_significativo else 'No se rechaza'} "
    interpretacion += "la hipótesis nula" + (" (hay diferencias significativas)" if es_significativo else "")

    print(f'Categoria de exito: {success_category}')
    print(interpretacion)

    return p_value

In [ ]:
results = compare_proportions(df, 
                              group_column='version',
                              category_column='retention_7',
                              success_category=True)

## Prueba t para dos muestras independientes

Los promedios también son una métrica comúnmente utilizada en las pruebas A/B. Comparemos las rondas de juego en promedio por cada grupo en el experimento.

In [ ]:
# Estadística descriptiva
df.groupby('version').agg({'sum_gamerounds':'mean'})

Consideraciones previas para realizar una prueba t:  
- la variable debe tener escala de medición de intervalo o de razón
- la variable debe tener una distribución normal o la muestra debe ser grande (mayor a 30)
- las observaciones deben ser independientes.

Planteamiento de hipótesis:  
$H_0: \mu_1 = \mu_2$  
$H_1: \mu_1 \neq \mu_2$

In [ ]:
from scipy.stats import ttest_ind

def compare_means(data, group_column, measure_column,
                  group1=None, group2=None,
                  equal_var: bool = True,
                  alpha = 0.05):
    """
    Realiza e interpreta una prueba t bilateral para comparar medias entre grupos.
    Args:
        data: DataFrame con los datos
        group_column: Nombre de la columna que identifica los grupos
        measure_column: Nombre de la columna con la variable a comparar
        group1: Primer grupo a comparar (si es None, usa el primero encontrado)
        group2: Segundo grupo a comparar (si es None, usa el segundo encontrado)
        equal_var: Si es True, asume varianzas iguales (t de Student)
                   Si es False, no asume varianzas iguales (t de Welch)
        alpha: Nivel de significancia (default: 0.05)
    Returns:
        p-value
    """
    # Identificar grupos si no se especifican
    grupos = data[group_column].unique()    # Obtener valores únicos
    if len(grupos) != 2:
        raise ValueError(f"La columna {group_column} debe tener exactamente dos grupos")  
    group1 = group1 if group1 is not None else grupos[0]
    group2 = group2 if group2 is not None else grupos[1]
    
    # Obtener datos de cada grupo
    datos_group1 = data[data[group_column] == group1][measure_column]
    datos_group2 = data[data[group_column] == group2][measure_column]
    
    # Calcular estadísticos descriptivos
    stats = pd.DataFrame({
        'n': [len(datos_group1), len(datos_group2)],
        'media': [datos_group1.mean(), datos_group2.mean()],
        'std': [datos_group1.std(), datos_group2.std()]}, 
        index=[group1, group2])
    
    # Realizar prueba t
    t_stat, p_value = ttest_ind(datos_group1, datos_group2, equal_var=equal_var)
    
    # Preparar interpretación
    es_significativo = p_value < alpha
    interpretacion = (f"{'Prueba t de Student' if equal_var else 'Prueba t de Welch'}\n"
                      f"Estadístico t: {t_stat:.4f}\n"
                      f"Valor p: {p_value:.4f}\n\n"
                      "Estadísticos descriptivos:\n")
    
    for grupo in [group1, group2]:
        n = stats.loc[grupo, 'n']
        media = stats.loc[grupo, 'media']
        std = stats.loc[grupo, 'std']
        interpretacion += f"{grupo}: media={media:.2f}, std={std:.2f}, n={n}\n"
    
    interpretacion += f"\nConclusión: {'Se rechaza' if es_significativo else 'No se rechaza'} "
    interpretacion += "la hipótesis nula" + (" (hay diferencias significativas entre las medias)" 
                                           if es_significativo else "")
    
    print(interpretacion)
    return p_value

In [ ]:
resultados = compare_means(df, 'version', 'sum_gamerounds')

El p-valor reportado corresponde a una prueba bilateral, es decir, cuando no se anticipa si la diferencia será positiva o negativa

Si la muestra es pequeña (p<30) se requiere evaluar la normalidad en la distribución de los datos. Existen varias pruebas para evaluar la normalidad de los datos. La prueba Shapiro-Wilk es una de las más utilizadas.  
- Hipótesis nula (H0): Los datos provienen de una distribución normal  
- Hipótesis alternativa (H1): Los datos no provienen de una distribución normal.  

## Prueba de Levene
La prueba de Levene de homogeneidad de varianzas evalúa si la varianza es la misma para los diferentes grupos. Se debe revisar el p-valor (significancia), si este es mayor que 0.05 entonces no rechaza la hipótesis nula de igualdad de varianzas (es decir, no se está violando el supuesto de homogeneidad de varianzas)

In [ ]:
from scipy.stats import levene

def compare_variances(data, group_column, measure_column, groups=None, alpha=0.05):
    """
    Realiza e interpreta una prueba de Levene para comparar varianzas entre múltiples grupos.   
    Args:
        data: DataFrame con los datos
        group_column: Nombre de la columna que identifica los grupos
        measure_column: Nombre de la columna con la variable a comparar
        groups: Lista de grupos a comparar (si es None, usa todos los grupos encontrados)
        alpha: Nivel de significancia (default: 0.05)
    Returns:
        p-value
    """
    # Identificar grupos si no se especifican
    grupos_disponibles = data[group_column].unique()
    if len(grupos_disponibles) < 2:
        raise ValueError(f"La columna {group_column} debe tener al menos dos grupos")
    
    # Usar grupos especificados o todos los disponibles
    grupos = groups if groups is not None else grupos_disponibles
    
    # Verificar que todos los grupos especificados existen
    grupos_invalidos = set(grupos) - set(grupos_disponibles)
    if grupos_invalidos:
        raise ValueError(f"Grupos no encontrados en los datos: {grupos_invalidos}")
    
    # Obtener datos de cada grupo
    datos_grupos = [data[data[group_column] == grupo][measure_column] for grupo in grupos]
    
    # Calcular estadísticas por grupo
    stats = pd.DataFrame({
        'n': [len(datos) for datos in datos_grupos],
        'varianza': [datos.var() for datos in datos_grupos],
        'desv_std': [datos.std() for datos in datos_grupos]
    }, index=grupos)
    
    # Realizar prueba de Levene
    stat, p_value = levene(*datos_grupos)
    
    # Preparar interpretación
    es_significativo = p_value < alpha
    interpretacion = (f"Prueba de Levene para {len(grupos)} grupos\n"
                     f"Estadístico de Levene: {stat:.4f}\n"
                     f"Valor p: {p_value:.4f}\n\n"
                     "Estadísticas por grupo:\n")
    
    # Añadir estadísticas de cada grupo
    for grupo in grupos:
        n = stats.loc[grupo, 'n']
        var = stats.loc[grupo, 'varianza']
        std = stats.loc[grupo, 'desv_std']
        interpretacion += f"{grupo}: n={n}, varianza={var:.2f}, desv. estándar={std:.2f}\n"
    
    # Añadir conclusión
    interpretacion += f"\nConclusión: {'Se rechaza' if es_significativo else 'No se rechaza'} "
    interpretacion += "la hipótesis nula de igualdad de varianzas"
    interpretacion += (" (hay diferencias significativas en las varianzas entre los grupos)" 
                      if es_significativo 
                      else "\n (no hay evidencia de diferencias significativas en las varianzas)")
    
    print(interpretacion)
    return p_value

In [ ]:
resultado = compare_variances(df, 'version', 'sum_gamerounds')

## Prueba t de Welch 
Si hay evidencia estadística de que las varianzas son diferentes, se recomienda hacer una prueba t de Welch (para varianzas diferentes)

In [ ]:
resultados = compare_means(df, 'version', 'sum_gamerounds', equal_var=False )

## Prueba Mann-Whitney
Una alternativa no paramétrica a la prueba t es la prueba de Mann-Whitney. Esto es recomendable cuando no se cumple el supuesto de normalidad de los datos.

In [ ]:
from scipy.stats import mannwhitneyu

def compare_distributions(data, group_column, measure_column,
                          group1=None, group2=None, alpha = 0.05):
    """
    Realiza e interpreta una prueba de Mann-Whitney para comparar distribuciones.
    Args:
        data: DataFrame con los datos
        group_column: Nombre de la columna que identifica los grupos
        measure_column: Nombre de la columna con la variable a comparar
        group1: Primer grupo a comparar (si es None, usa el primero encontrado)
        group2: Segundo grupo a comparar (si es None, usa el segundo encontrado)
        alpha: Nivel de significancia (default: 0.05)
    Returns:
        valor p de la prueba
    """
    # Identificar grupos si no se especifican
    grupos = data[group_column].unique()
    if len(grupos) < 2:
        raise ValueError(f"La columna {group_column} debe tener al menos dos grupos")
    group1 = group1 if group1 is not None else grupos[0]
    group2 = group2 if group2 is not None else grupos[1]
    
    # Obtener datos de cada grupo
    datos_group1 = data[data[group_column] == group1][measure_column]
    datos_group2 = data[data[group_column] == group2][measure_column]
    
    # Calcular estadísticos descriptivos
    stats = pd.DataFrame({
        'n': [len(datos_group1), len(datos_group2)],
        'mediana': [datos_group1.median(), datos_group2.median()],
        'rango_iq': [datos_group1.quantile(0.75) - datos_group1.quantile(0.25),
                    datos_group2.quantile(0.75) - datos_group2.quantile(0.25)]
    }, index=[group1, group2])
    
    # Realizar prueba de Mann-Whitney
    statistic, p_value = mannwhitneyu(datos_group1, datos_group2)
    
    # Preparar interpretación
    es_significativo = p_value < alpha
    interpretacion = (f"Estadístico U: {statistic:.4f}\n"
                      f"Valor p: {p_value:.4f}\n\n"
                      "Estadísticos descriptivos:\n")
    
    for grupo in [group1, group2]:
        n = stats.loc[grupo, 'n']
        mediana = stats.loc[grupo, 'mediana']
        rango = stats.loc[grupo, 'rango_iq']
        interpretacion += f"{grupo}: mediana={mediana:.2f}, rango_IQ={rango:.2f}, n={n}\n"
    
    interpretacion += f"\nConclusión: {'Se rechaza' if es_significativo else 'No se rechaza'} "
    interpretacion += "la hipótesis nula" + (" (hay diferencias significativas en la distribución)" 
                                           if es_significativo else "")
    
    print(interpretacion)
    return p_value

In [ ]:
resultados = compare_distributions(df, 'version', 'sum_gamerounds')

## Ejercicio
Utiliza la base de datos "enigh_2020" para evaluar si ___________ por hogar en promedio es diferente entre los hogares donde el jefe de familia es hombre en comparación con los hogares donde el jefe de familia es mujer.

- Cargar los datos y librerías, explorar los datos
- Evaluar supuestos: igualdad de varianzas y normalidad (no se requiere en este caso)
- Realizar la prueba correspondiente

## Referencias
https://roirevolution.com/blog/why-ab-testing-could-save-your-marketing-strategy/